In [1]:
import pandas as pd
import numpy as np
import math

# 1. CONFIGURATION

In [2]:
class Config:
    GOLD_FILE = 'gold_interactions.csv'
    
    USER_BASED_FILE = 'user_based_results.csv'
    ITEM_BASED_FILE = 'item_based_results.csv'
    
    # Evaluation constraints
    K_PRECISION = [5, 10]
    NDCG_K = 10

# 2. METRICS IMPLEMENTATION

In [3]:
def calculate_precision_at_k(recommended_list, gold_set, k):
    top_k = recommended_list[:k]
    if not top_k:
        return 0.0
    relevant_cnt = sum(1 for item in top_k if item in gold_set)
    return relevant_cnt / k

def calculate_ndcg(recommended_list, gold_ratings_dict, k):
    dcg = 0.0
    top_k = recommended_list[:k]
    
    for i, item in enumerate(top_k):
        rel = gold_ratings_dict.get(item, 0.0)
        gain = (2**rel - 1)
        discount = math.log2((i + 1) + 1)
        dcg += gain / discount
        
    # Calculate Ideal DCG
    ideal_ratings = sorted(gold_ratings_dict.values(), reverse=True)[:k]
    idcg = 0.0
    for i, rel in enumerate(ideal_ratings):
        gain = (2**rel - 1)
        discount = math.log2((i + 1) + 1)
        idcg += gain / discount
        
    if idcg == 0: return 0.0
    return dcg / idcg

def calculate_spearman(recommended_list, gold_ratings_dict):
    xs = [] 
    ys = []
    for rank_idx, item in enumerate(recommended_list):
        if item in gold_ratings_dict:
            xs.append(rank_idx + 1)
            ys.append(gold_ratings_dict[item])
            
    n = len(xs)
    if n < 2: return 0.0
    
    x = np.array(xs)
    y = np.array(ys)
    
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    
    numerator = np.sum((x - x_mean) * (y - y_mean))
    denom_x = np.sqrt(np.sum((x - x_mean)**2))
    denom_y = np.sqrt(np.sum((y - y_mean)**2))
    
    if denom_x == 0 or denom_y == 0: return 0.0
    return numerator / (denom_x * denom_y)

# 3. EVALUATION RUNNER

In [4]:
def load_gold_data():
    print("Loading Gold Data...")
    df = pd.read_csv(Config.GOLD_FILE)
    gold_data = {}
    for _, row in df.iterrows():
        uid = int(row['user_id'])
        bid = int(row['book_id'])
        rating = float(row['rating'])
        
        if uid not in gold_data: gold_data[uid] = {}
        gold_data[uid][bid] = rating
    return gold_data

def evaluate_file(filename, gold_data, algorithm_name):
    print(f"\n--- Evaluating {algorithm_name} ---")
    
    try:
        df = pd.read_csv(filename)
    except FileNotFoundError:
        print(f"Error: Could not find {filename}. Run main.py first.")
        return
    
    metrics = {'P@5': [], 'P@10': [], 'NDCG': [], 'Spearman': []}
    
    for _, row in df.iterrows():
        uid = int(row['user_id'])
        
        if pd.isna(row['recommendations']):
            recs = []
        else:
            # Handle potential float/string formatting issues in CSV
            recs_str = str(row['recommendations']).strip()
            if not recs_str:
                recs = []
            else:
                recs = [int(float(x)) for x in recs_str.split()]
            
        user_gold_dict = gold_data.get(uid, {})
        user_gold_set = set(user_gold_dict.keys())
        
        if not user_gold_dict: continue
            
        metrics['P@5'].append(calculate_precision_at_k(recs, user_gold_set, 5))
        metrics['P@10'].append(calculate_precision_at_k(recs, user_gold_set, 10))
        metrics['NDCG'].append(calculate_ndcg(recs, user_gold_dict, Config.NDCG_K))
        metrics['Spearman'].append(calculate_spearman(recs, user_gold_dict))
        
    print(f"Results for {algorithm_name}:")
    print(f"Mean P@5:      {np.mean(metrics['P@5']):.4f}")
    print(f"Mean P@10:     {np.mean(metrics['P@10']):.4f}")
    print(f"Mean NDCG:     {np.mean(metrics['NDCG']):.4f}")
    print(f"Mean Spearman: {np.mean(metrics['Spearman']):.4f}")

# 4. MAIN

In [5]:
def main():
    gold_data = load_gold_data()
    
    # Evaluate Method 1: User-Based
    evaluate_file(Config.USER_BASED_FILE, gold_data, "User-Based CF")
    
    # Evaluate Method 2: Item-Based
    evaluate_file(Config.ITEM_BASED_FILE, gold_data, "Item-Based CF")
    
    print("\n***Evaluation Complete.")

if __name__ == "__main__":
    main()

Loading Gold Data...

--- Evaluating User-Based CF ---
Results for User-Based CF:
Mean P@5:      0.0384
Mean P@10:     0.0298
Mean NDCG:     0.0384
Mean Spearman: -0.0026

--- Evaluating Item-Based CF ---
Results for Item-Based CF:
Mean P@5:      0.0056
Mean P@10:     0.0060
Mean NDCG:     0.0055
Mean Spearman: -0.0040

***Evaluation Complete.
